# 학습데이터 준비 (전처리 과정)

- 예제로 사용된 데이터는 `AI 허브`에 공개된 데이터를 사용했습니다.

  - 출처 : https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=realm&dataSetSn=98
  - 참고 : https://openai.com/blog/gpt-3-5-turbo-fine-tuning-and-api-updates

In [ ]:
import json

def convert_data(input_file, output_file):
    with open(input_file, 'r') as f:
        data = json.load(f)

    grouped_data = {}
    for item in data:
        dialogue_number = item["대화셋일련번호"]

        if dialogue_number not in grouped_data:
            grouped_data[dialogue_number] = [{"role": "system", "content": "이것은 고객센터에서 대중교통에 대한 고객 질문에 답변하는 대화입니다."}]

        role = "assistant" if item["화자"] == "상담사" else "user"
        question = item["고객질문(요청)"] if role == "user" else item["상담사질문(요청)"]
        answer = item["고객답변"] if role == "user" else item["상담사답변"]

        if question:
            grouped_data[dialogue_number].append({"role": role, "content": question})
        if answer:
            grouped_data[dialogue_number].append({"role": role, "content": answer})

    with open(output_file, 'w', encoding='utf-8') as f:
        for key, val in grouped_data.items():
            f.write(json.dumps({"messages": val}, ensure_ascii=False) + "\n")


convert_data('민원(콜센터) 질의응답_다산콜센터_대중교통 안내_Training.json', '민원(콜센터) 질의응답_다산콜센터_대중교통 안내_Training.jsonl')

# 데이터셋 업로드
- 데이터셋은 OpenAI에 업로드해야 학습에 이용 할 수 있습니다.

In [ ]:
import os
import openai

openai.api_key = 'YOUR_API_KEY'
openai.File.create(
  file=open("민원(콜센터) 질의응답_다산콜센터_대중교통 안내_Training.jsonl", "rb"),
  purpose='fine-tune'
)

# 파인튜닝 job 생성
- 파인튜닝 job을 생성하고 잠시 기다리면 파인튜닝이 시작됩니다.
- 데이터셋 크기와 epoch 수에 따라 시간이 오래 걸릴 수 있습니다.
- 예제로 사용된 다산 콜센터 상담 내용은 3 epoch 진행되었고 대략 50분 정도의 시간이 필요합니다. 


In [ ]:
openai.FineTuningJob.create(training_file="YOUR_FILE", model="gpt-3.5-turbo")

In [ ]:
# List 10 fine-tuning jobs
openai.FineTuningJob.list(limit=10)

# Retrieve the state of a fine-tune
# openai.FineTuningJob.retrieve("YOUR_JOB")

# Cancel a job
# openai.FineTuningJob.cancel("YOUR_JOB")

# List up to 10 events from a fine-tuning job
# openai.FineTuningJob.list_events(id="YOUR_JOB", limit=10)

# Delete a fine-tuned model (must be an owner of the org the model was created in)
# openai.Model.delete("YOUR_JOB")

# 파인 튜닝된 모델을 사용해 질의하기

In [ ]:
completion = openai.ChatCompletion.create(
  model="YOUR_MODEL",
  messages=[
    {"role": "system", "content": "이것은 고객센터에서 대중교통에 대한 고객 질문에 답변하는 대화입니다."},
    {"role": "user", "content": "야탑역에서 판교역가는 버스를 알려주세요"}
  ]
)
print(completion.choices[0].message)